Implementing and querying stored procedures

In [2]:
# Task 1: Establish a connection 

from mysql.connector.pooling import  MySQLConnectionPool
from mysql.connector import Error
 
dbconfig={"database":"little_lemon_db", "user":"root", "password":""}

try:
    pool_a = MySQLConnectionPool(pool_name="pool_a", pool_size = 2, **dbconfig)
except Error as er:
    print("Error!")
    print("Code:", er.errno)
    print("Message:", er.msg)

print("-----Pool created-----")

conn = pool_a.get_connection()
cursor = conn.cursor()

print("-----Cursor created-----")

# Task 2: Implement a stored procedure called PeakHours

cursor.execute("DROP PROCEDURE IF EXISTS PeakHours;")
create_peakhours = """
    CREATE PROCEDURE PeakHours() 
    BEGIN 
        SELECT 
            HOUR(BookingSlot) AS BookingSlotHour, 
            COUNT(BookingID) AS NumberOfBookings 
        FROM Bookings
        GROUP BY BookingSlotHour 
        ORDER BY NumberOfBookings DESC;
    END 
"""
cursor.execute(create_peakhours)
conn.commit()
print("-----PeakHours procedure created-----")

print("-----Calling PeakHours procedure-----")

cursor.callproc('PeakHours')
results = next(cursor.stored_results())
dataset = results.fetchall()
column_names = [col[0] for col in results.description]
print(column_names)
for row in dataset:
    print(row)

# Task 3: Implement a stored procedure GuestStatus

cursor.execute("DROP PROCEDURE IF EXISTS GuestStatus;")
create_gueststatus = """
    CREATE PROCEDURE GuestStatus()
    BEGIN
        SELECT 
            CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS FullName, 
            CASE 
                WHEN e.Role IN ('Manager', 'Assistant Manager') THEN 'Ready to pay'
                WHEN e.Role = 'Head Chef' THEN 'Ready to serve'
                WHEN e.Role = 'Assistant Chef' THEN 'Preparing Order'
                WHEN e.Role = 'Head Waiter' THEN 'Order served'
                ELSE 'Status not available'
            END AS Status
        FROM Bookings AS b 
        LEFT JOIN Employees AS e ON b.EmployeeID = e.EmployeeID;
    END
"""
cursor.execute(create_gueststatus)
conn.commit()
print("-----GuestStatus procedure created-----")

print("-----Calling GuestStatus procedure-----")

cursor.callproc('GuestStatus')
results = next(cursor.stored_results())
dataset = results.fetchall()
column_names = [col[0] for col in results.description]
print(column_names)
for row in dataset:
    print(row)

conn.close()

print("-----Connection closed-----")

-----Pool created-----
-----Cursor created-----
-----PeakHours procedure created-----
-----Calling PeakHours procedure-----
['BookingSlotHour', 'NumberOfBookings']
(19, 4)
(18, 2)
(15, 1)
(17, 1)
(20, 1)
-----GuestStatus procedure created-----
-----Calling GuestStatus procedure-----
['FullName', 'Status']
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')
('Anees Java', 'Status not available')
('Bald Vin', 'Status not available')
('Jay Kon', 'Status not available')
-----Connection closed-----
